<a href="https://colab.research.google.com/github/KhoiDOO/Practice/blob/main/Recommendation%20System/4.%20Retrieval_with_Sequential_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 3.3 MB/s 
     |████████████████████████████████| 462 kB 29.4 MB/s 
     |████████████████████████████████| 4.2 MB 5.1 MB/s 


In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
!wget -nc https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/recommendation/ml/data/example_generation_movielens.py
!python -m example_generation_movielens  --data_dir=data/raw  --output_dir=data/examples  --min_timeline_length=3  --max_context_length=10  --max_context_movie_genre_length=10  --min_rating=2  --train_data_fraction=0.9  --build_vocabs=False

--2022-03-01 14:22:42--  https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/recommendation/ml/data/example_generation_movielens.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18040 (18K) [text/plain]
Saving to: ‘example_generation_movielens.py’

example_generation_ 100%[===================>]  17.62K  --.-KB/s    in 0.001s  

2022-03-01 14:22:42 (31.8 MB/s) - ‘example_generation_movielens.py’ saved [18040/18040]

2022-03-01 14:22:45.780952: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I0301 14:22:45.781157 140713278101376 example_generation_movielens.py:460] Downloading and extracting data.
5931008/5917549 [==============================] - 0s 

In [5]:
# 0 : {
#   features: {
#     feature: {
#       key  : "context_movie_id"
#       value: { int64_list: { value: [ 1124, 2240, 3251, ..., 1268 ] } }
#     }
#     feature: {
#       key  : "context_movie_rating"
#       value: { float_list: {value: [ 3.0, 3.0, 4.0, ..., 3.0 ] } }
#     }
#     feature: {
#       key  : "context_movie_year"
#       value: { int64_list: { value: [ 1981, 1980, 1985, ..., 1990 ] } }
#     }
#     feature: {
#       key  : "context_movie_genre"
#       value: { bytes_list: { value: [ "Drama", "Drama", "Mystery", ..., "UNK" ] } }
#     }
#     feature: {
#       key  : "label_movie_id"
#       value: { int64_list: { value: [ 3252 ] }  }
#     }
#   }
# }

In [6]:
train_filename = "./data/examples/train_movielens_1m.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "./data/examples/test_movielens_1m.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

feature_description = {
    'context_movie_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_movie_rating': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_movie_year': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(1980, 10)),
    'context_movie_genre': tf.io.FixedLenFeature([10], tf.string, default_value=np.repeat("Drama", 10)),
    'label_movie_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)

train_ds = train.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})

test_ds = test.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})

for x in train_ds.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'context_movie_id': array([b'973', b'904', b'908', b'1248', b'1283', b'910', b'922', b'2664',
       b'1104', b'3736'], dtype=object),
 'label_movie_id': array([b'1250'], dtype=object)}


In [7]:
movies = tfds.load("movielens/1m-movies", split='train')
movies = movies.map(lambda x: x["movie_id"])
movie_ids = movies.batch(1_000)
unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3883 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/1m-movies/0.1.0.incompleteLBYLDB/movielens-train.tfrecord*...:  …

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/1m-movies/0.1.0. Subsequent calls will reuse this data.


In [8]:
embedding_dimension = 32

query_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_movie_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension), 
    tf.keras.layers.GRU(embedding_dimension),
])

candidate_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension)
])

In [9]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(candidate_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

class Model(tfrs.Model):

    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model

        self._task = task

    def compute_loss(self, features, training=False):
        watch_history = features["context_movie_id"]
        watch_next_label = features["label_movie_id"]

        query_embedding = self._query_model(watch_history)       
        candidate_embedding = self._candidate_model(watch_next_label)

        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)

In [10]:
model = Model(query_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [11]:
cached_train = train_ds.shuffle(10_000).batch(12800).cache()
cached_test = test_ds.batch(2560).cache()

In [12]:
model.fit(cached_train, epochs=3)

Epoch 1/3
67/67 [==============================] - 225s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 107940.9007 - regularization_loss: 0.0000e+00 - total_loss: 107940.9007
Epoch 2/3
67/67 [==============================] - 176s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 100977.7027 - regularization_loss: 0.0000e+00 - total_loss: 100977.7027
Epoch 3/3
67/67 [==============================] - 164s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - 

In [13]:
model.evaluate(cached_test, return_dict=True)

37/37 [==============================] - 48s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0147 - factorized_top_k/top_5_categorical_accuracy: 0.0789 - factorized_top_k/top_10_categorical_accuracy: 0.1375 - factorized_top_k/top_50_categorical_accuracy: 0.3771 - factorized_top_k/top_100_categorical_accuracy: 0.5099 - loss: 15455.2240 - regularization_loss: 0.0000e+00 - total_loss: 15455.2240


{'factorized_top_k/top_100_categorical_accuracy': 0.509898841381073,
 'factorized_top_k/top_10_categorical_accuracy': 0.13745349645614624,
 'factorized_top_k/top_1_categorical_accuracy': 0.014722971245646477,
 'factorized_top_k/top_50_categorical_accuracy': 0.3770829141139984,
 'factorized_top_k/top_5_categorical_accuracy': 0.07894540578126907,
 'loss': 9328.5283203125,
 'regularization_loss': 0,
 'total_loss': 9328.5283203125}